In [109]:
from test_eos import *
import astro_const as ac
from structure import stellar_derivatives, central_values, lengthscales, integrate, pressure_guess
from eos import *

In [110]:
Pc = pressure_guess(ac.Msun, 2)
m,r,p = integrate(Pc, 1e-4,2e-6,.05,2)
print(m[-1],r[-1],p[-1])

6.017575396667514e+30 6094895.241167408 2.648592992998796e+18


## We used Msun, yet got 6e30 back for mass, Pguess is bad, we must correct it using bisect function below

In [111]:
#Function to use with bisect
def f(Pc,Mwant,delta_m,eta,xi,mue):
    m,r,p = integrate(Pc,delta_m,eta,xi,mue)
    M = m[-1]
    return(M-Mwant)

In [121]:
#Using bisect to get correction factor for Pguess
from scipy import optimize
masses = ac.Msun*np.arange(0.1,1.1,0.1)
Pcs = np.zeros(10)
Pguesses = np.zeros(10)

#parameters
delta_m = 1e-4
eta = 5e-7 
xi = .05
mue = 2

for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pguesses[i] = Pguess
    Pc = optimize.bisect(f,Pguess/100,Pguess,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

In [122]:
#Correction factor, divid Pguess by this number
np.mean(Pguesses/Pcs) 

40.093381320405605

In [123]:
#Implimenting correction factor
Pc = pressure_guess(ac.Msun, 2)/np.mean(Pguesses/Pcs)
m,r,p = integrate(Pc, 1e-4,5e-7,.05,2)
print(m[-1],r[-1],p[-1])

1.9884100319691045e+30 8842493.814427385 1.6607752356574852e+16


In [124]:
ac.Msun

1.988409870698051e+30

# Masses are closer now.

In [125]:
import pandas as pd

In [126]:
R = np.zeros(10)
P = np.zeros(10)
Pmks = np.zeros(10)
rhomks = np.zeros(10)
rho = np.zeros(10)

for i,j in enumerate(np.arange(.1,1.1,.1)):
    Pc = pressure_guess(ac.Msun*j, 2)/np.mean(Pguesses/Pcs)
    m,r,p = integrate(Pc, 1e-4,5e-7,.05,2)
    R[i] = r[-1]
    P[i] = Pc
    Pmks[i] = Pc/(ac.G*(ac.Msun*j)**2*R[i]**(-4))
    rhomks[i] = density(P[i],2)
    rho[i] = rhomks[i]/((3*ac.Msun*j)/(ac.fourpi*R[i]**3))

In [127]:
diction = {"M/Msun":np.arange(.1,1.1,.1), "R/Rsun":R/6.96e8,"Pc": P,
          "Pc/stuff":Pmks, "rhoc": rhomks, "rho": rho}

table = pd.DataFrame(diction)
table

,M/Msun,R/Rsun,Pc,Pc/stuff,rhoc,rho
0,0.1,0.027370,1.518007e+19,0.757552,4.063113e+07,5.917101
1,0.2,0.021724,1.530055e+20,0.757563,1.625245e+08,5.917164
2,0.3,0.018978,5.911229e+20,0.757589,3.656802e+08,5.917318
3,0.4,0.017242,1.542198e+21,0.757574,6.500980e+08,5.917227
4,0.5,0.016007,3.244693e+21,0.757675,1.015778e+09,5.917817
5,0.6,0.015063,5.958146e+21,0.757600,1.462721e+09,5.917378
6,0.7,0.014308,9.960178e+21,0.757573,1.990925e+09,5.917220
7,0.8,0.013685,1.554439e+22,0.757585,2.600392e+09,5.917290
8,0.9,0.013159,2.301876e+22,0.757624,3.291121e+09,5.917519
9,1.0,0.012705,3.270446e+22,0.757685,4.063113e+09,5.917876


# Rubric:

Pc/stuff = .77

rho = 5.99